In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
import matplotlib.image as mpimg 

2023-12-10 20:31:17.490945: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Read X_train
with open("train_test/X_train.pkl", "rb") as file:
    X_train = pickle.load(file)

# Read X_test
with open("train_test/X_test.pkl", "rb") as file:
    X_test = pickle.load(file)

# Read y_train
with open("train_test/y_train.pkl", "rb") as file:
    y_train = pickle.load(file)

# Read y_test
with open("train_test/y_test.pkl", "rb") as file:
    y_test = pickle.load(file)


In [3]:
xy = 200

In [4]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
])

2023-12-10 20:31:22.833545: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
model_cnn = tf.keras.Sequential([
    data_augmentation,
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), input_shape=(xy, xy, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),

    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model_cnn.compile(
    loss= tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

In [6]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

history_cnn =  model_cnn.fit(X_train, y_train, epochs=10, callbacks=[early_stopping])

Epoch 1/10
87/87 [==============================] - 43s 485ms/step - loss: 1.6256 - accuracy: 0.3385
Epoch 2/10
87/87 [==============================] - 41s 475ms/step - loss: 0.8713 - accuracy: 0.3616
Epoch 3/10
87/87 [==============================] - 40s 465ms/step - loss: 0.7287 - accuracy: 0.3549
Epoch 4/10
21/87 [======>.......................] - ETA: 30s - loss: 0.6473 - accuracy: 0.3378

In [ ]:
test_loss_cnn, test_acc_cnn = model_cnn.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc_cnn)

In [ ]:
import pickle

# Pickle the model
with open("model_cnn.pkl", "wb") as file:
    pickle.dump(model_cnn, file)

In [ ]:
probability_model = tf.keras.Sequential([model_cnn, tf.keras.layers.Softmax()])

predictions = probability_model.predict(X_test)

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
y_test[0]

In [ ]:
test_img = X_test[0]
print(test_img.shape)
plt.imshow(test_img)

In [ ]:
epoch = len(history_cnn.history.get('loss',[]))

# Draw Model Accuracy
plt.figure(2,figsize=(6,4))
plt.plot(range(epoch),history_cnn.history.get('accuracy'))
#plt.plot(range(epoch),training_history.history.get('val_acc'))
plt.xlabel('# Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.grid(True)
plt.legend(['train','validation'],loc=4)
plt.style.use(['classic'])

# Draw Model Loss
plt.figure(1,figsize=(6,4))
plt.plot(range(epoch),history_cnn.history.get('loss'))
#plt.plot(range(epoch),training_history.history.get('val_loss'))
plt.xlabel('# Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.grid(True)
plt.legend(['train','validation'], loc=4)
plt.style.use(['classic'])